In [61]:
import pandas as pd

df = pd.read_csv("../data/clean/cleaned_combined_reviews.csv")

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98165 entries, 0 to 98164
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_name    98165 non-null  object
 1   review_text  98165 non-null  object
 2   rating       98165 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 2.2+ MB


In [62]:
df["user_name"] = df["user_name"].astype(str)
df["review_text"] = df["review_text"].astype(str)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98165 entries, 0 to 98164
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_name    98165 non-null  object
 1   review_text  98165 non-null  object
 2   rating       98165 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 2.2+ MB


### Review length of text

The review length would let us categorise very short reviews is irrelevant. Long reviews could potentially be rants.

In [63]:
# extract lengths of each review
df["review_length"] = df["review_text"].apply(len)
df.head(10)

,user_name,review_text,rating,review_length
0,Amber Thibeault,Andrea is amazing. Our dog loves her and she a...,5,95
1,Esther,Andrea does a wonderful job with our wild Pr...,5,90
2,Bob Barrett,Never called back,1,17
3,Luz Quiles,They don't answer the phones,3,28
4,Tim Sanderson,Limited information on the website,3,34
5,Ellen Nastir,Leigh-Ann is an incredibly creative facilitato...,5,251
6,Jinnie Lee Schmid,Leigh Ann Rodgers is THE undisputed expert in ...,5,649
7,Wanda Walker,Leigh Ann Rodgers is a great collaborator and ...,5,251
8,Heather Clarke,I really appreciate all the wisdom and experie...,5,188
9,Cheryle Maurer,Leigh Ann’s masterful facilitation and engagem...,5,171


### Tokenize review text
This is for training the ML model.

In [64]:
import spacy

nlp = spacy.load("en_core_web_sm")

# made a small dataset first
df2 = df.copy().head(200)

tokens_list = []
for text in df2["review_text"]:
    doc = nlp(text)
    tokens_list.append([token.text for token in doc])

df2["tokens"] = tokens_list
df2.head(10)

,user_name,review_text,rating,review_length,tokens
0,Amber Thibeault,Andrea is amazing. Our dog loves her and she a...,5,95,"[Andrea, is, amazing, ., Our, dog, loves, her,..."
1,Esther,Andrea does a wonderful job with our wild Pr...,5,90,"[Andrea, does, a, wonderful, , job, , with, ..."
2,Bob Barrett,Never called back,1,17,"[Never, called, back]"
3,Luz Quiles,They don't answer the phones,3,28,"[They, do, n't, answer, the, phones]"
4,Tim Sanderson,Limited information on the website,3,34,"[Limited, information, on, the, website]"
5,Ellen Nastir,Leigh-Ann is an incredibly creative facilitato...,5,251,"[Leigh, -, Ann, is, an, incredibly, creative, ..."
6,Jinnie Lee Schmid,Leigh Ann Rodgers is THE undisputed expert in ...,5,649,"[Leigh, Ann, Rodgers, is, THE, undisputed, exp..."
7,Wanda Walker,Leigh Ann Rodgers is a great collaborator and ...,5,251,"[Leigh, Ann, Rodgers, is, a, great, collaborat..."
8,Heather Clarke,I really appreciate all the wisdom and experie...,5,188,"[I, really, appreciate, all, the, wisdom, and,..."
9,Cheryle Maurer,Leigh Ann’s masterful facilitation and engagem...,5,171,"[Leigh, Ann, ’s, masterful, facilitation, and,..."


### Sentiment Analysis
Analyse the emotions in each review. Very negative reviews could potentially be rants.

In [65]:
from transformers import pipeline


#Load the pre-trained sentiment analysis pipeline
sentiment_analyzer = pipeline("sentiment-analysis")
labels = []
scores = []

for text in df2["review_text"]:
    result = sentiment_analyzer(text)[0]  # returns list of dicts
    label = result['label']
    score = result['score']
    labels.append(label)
    scores.append(score)

df2["sentiment"] = labels
df2["confidence score"] = scores
df2.head(10)

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use mps:0


,user_name,review_text,rating,review_length,tokens,sentiment,confidence score
0,Amber Thibeault,Andrea is amazing. Our dog loves her and she a...,5,95,"[Andrea, is, amazing, ., Our, dog, loves, her,...",POSITIVE,0.999887
1,Esther,Andrea does a wonderful job with our wild Pr...,5,90,"[Andrea, does, a, wonderful, , job, , with, ...",POSITIVE,0.999869
2,Bob Barrett,Never called back,1,17,"[Never, called, back]",NEGATIVE,0.974816
3,Luz Quiles,They don't answer the phones,3,28,"[They, do, n't, answer, the, phones]",NEGATIVE,0.999450
4,Tim Sanderson,Limited information on the website,3,34,"[Limited, information, on, the, website]",NEGATIVE,0.909106
5,Ellen Nastir,Leigh-Ann is an incredibly creative facilitato...,5,251,"[Leigh, -, Ann, is, an, incredibly, creative, ...",POSITIVE,0.999858
6,Jinnie Lee Schmid,Leigh Ann Rodgers is THE undisputed expert in ...,5,649,"[Leigh, Ann, Rodgers, is, THE, undisputed, exp...",POSITIVE,0.999863
7,Wanda Walker,Leigh Ann Rodgers is a great collaborator and ...,5,251,"[Leigh, Ann, Rodgers, is, a, great, collaborat...",POSITIVE,0.999833
8,Heather Clarke,I really appreciate all the wisdom and experie...,5,188,"[I, really, appreciate, all, the, wisdom, and,...",POSITIVE,0.999849
9,Cheryle Maurer,Leigh Ann’s masterful facilitation and engagem...,5,171,"[Leigh, Ann, ’s, masterful, facilitation, and,...",POSITIVE,0.999850


### All caps ratio
Some may use lots of capitalisation to indicate frustration in their reviews and they may be rants.

In [66]:
def all_caps_ratio(text):
    caps = 0
    for c in text:
        if c.isalpha() and c.isupper():
            caps += 1
    if len(text) == 0:
        return 0
    return (caps / len(text))

ratios = []

for text in df2["review_text"]:
    ratios.append(all_caps_ratio(text))

df2["all caps ratio"] = ratios

df2.head(10)

,user_name,review_text,rating,review_length,tokens,sentiment,confidence score,all caps ratio
0,Amber Thibeault,Andrea is amazing. Our dog loves her and she a...,5,95,"[Andrea, is, amazing, ., Our, dog, loves, her,...",POSITIVE,0.999887,0.031579
1,Esther,Andrea does a wonderful job with our wild Pr...,5,90,"[Andrea, does, a, wonderful, , job, , with, ...",POSITIVE,0.999869,0.044444
2,Bob Barrett,Never called back,1,17,"[Never, called, back]",NEGATIVE,0.974816,0.058824
3,Luz Quiles,They don't answer the phones,3,28,"[They, do, n't, answer, the, phones]",NEGATIVE,0.999450,0.035714
4,Tim Sanderson,Limited information on the website,3,34,"[Limited, information, on, the, website]",NEGATIVE,0.909106,0.029412
5,Ellen Nastir,Leigh-Ann is an incredibly creative facilitato...,5,251,"[Leigh, -, Ann, is, an, incredibly, creative, ...",POSITIVE,0.999858,0.015936
6,Jinnie Lee Schmid,Leigh Ann Rodgers is THE undisputed expert in ...,5,649,"[Leigh, Ann, Rodgers, is, THE, undisputed, exp...",POSITIVE,0.999863,0.023112
7,Wanda Walker,Leigh Ann Rodgers is a great collaborator and ...,5,251,"[Leigh, Ann, Rodgers, is, a, great, collaborat...",POSITIVE,0.999833,0.035857
8,Heather Clarke,I really appreciate all the wisdom and experie...,5,188,"[I, really, appreciate, all, the, wisdom, and,...",POSITIVE,0.999849,0.047872
9,Cheryle Maurer,Leigh Ann’s masterful facilitation and engagem...,5,171,"[Leigh, Ann, ’s, masterful, facilitation, and,...",POSITIVE,0.999850,0.017544


### Extract keywords
Find keywords that are usually found in advertisements to mark the reviews as advertisements.

In [67]:
import re

ad_keywords = [
    "promo", "discount", "offer", "buy now", "free", "click here", "visit",
    "limited time", "sale", "deal", "coupon", "special offer", "subscribe",
    "register now", "sign up", "exclusive", "order now", "save big",
    "hot deal", "shop now", "get it now", "today only", "claim your", "bonus"
]

def contains_ad_keywords(text):
    text_lower = text.lower()
    # ads will be marked with 1, the rest as 0
    return int(any(re.search(r'\b'+kw+r'\b', text_lower) for kw in ad_keywords))


ads = []
for text in df2["review_text"]:
    ad = contains_ad_keywords(text)
    ads.append(ad)

df2["ad"] = ads

ad_rows = df2[df2["ad"] == 1]


In [68]:
ad_rows

,user_name,review_text,rating,review_length,tokens,sentiment,confidence score,all caps ratio,ad
5,Ellen Nastir,Leigh-Ann is an incredibly creative facilitato...,5,251,"[Leigh, -, Ann, is, an, incredibly, creative, ...",POSITIVE,0.999858,0.015936,1
19,Sister King,"To whom this may concern,\nRecently, I was inj...",5,1044,"[To, whom, this, may, concern, ,, \n, Recently...",NEGATIVE,0.973565,0.035441,1
33,Charmian McKeown,Horrendous experience\n\nOrdered an expensive ...,1,1574,"[Horrendous, experience, \n\n, Ordered, an, ex...",NEGATIVE,0.999430,0.022236,1
69,Sosi Audain,Eric is epic! He taught music production to m...,5,163,"[Eric, is, epic, !, , He, taught, music, prod...",POSITIVE,0.999140,0.042945,1
109,Luz Brown,This review is long overdue. I called Five Or ...,5,916,"[This, review, is, long, overdue, ., I, called...",NEGATIVE,0.979283,0.016376,1
110,Philip Ardery,"Before finding Five or Free, I was struggling ...",5,1260,"[Before, finding, Five, or, Free, ,, I, was, s...",POSITIVE,0.960776,0.018254,1
111,Karen Colangelo,The team at Five or Free did a great job handl...,5,348,"[The, team, at, Five, or, Free, did, a, great,...",POSITIVE,0.999857,0.022989,1
112,Dana Kirby,I would actually give Five or Free 10 stars if...,5,958,"[I, would, actually, give, Five, or, Free, 10,...",POSITIVE,0.984782,0.019833,1
113,Lawrence Rice,"When I was preparing my house for sale, a home...",5,1623,"[When, I, was, preparing, my, house, for, sale...",POSITIVE,0.996676,0.021565,1
116,Ken Maclin,These guys are the ABSOLUTE BEST! The Five Or...,5,574,"[These, guys, are, the, ABSOLUTE, BEST, !, , ...",POSITIVE,0.999649,0.043554,1


In [69]:
with_features_path = "../data/with_features/with_features.csv"
df.to_csv(with_features_path, index=False)